In [ ]:
!unzip liver-ultrasound-detection.zip

Archive:  liver-ultrasound-detection.zip
  inflating: mapping.xlsx            
  inflating: sample_submission.csv   
  inflating: test/test/images/1.jpg  
  inflating: test/test/images/10.jpg  
  inflating: test/test/images/1000.jpg  
  inflating: test/test/images/10001.jpg  
  inflating: test/test/images/100077.jpg  
  inflating: test/test/images/100080.jpg  
  inflating: test/test/images/100105.jpg  
  inflating: test/test/images/100196.jpg  
  inflating: test/test/images/1002.jpg  
  inflating: test/test/images/100206.jpg  
  inflating: test/test/images/100212.jpg  
  inflating: test/test/images/100220.jpg  
  inflating: test/test/images/100227.jpg  
  inflating: test/test/images/100231.jpg  
  inflating: test/test/images/100296.jpg  
  inflating: test/test/images/1003.jpg  
  inflating: test/test/images/10039.jpg  
  inflating: test/test/images/1004.jpg  
  inflating: test/test/images/100452.jpg  
  inflating: test/test/images/100494.jpg  
  inflating: test/test/images/1005.jpg  
 

In [16]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from tqdm import tqdm
import pandas as pd
import numpy as np
from torchvision.ops import nms

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-05-29 21:49:00.962162: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 21:49:03.947218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 21:49:15.689292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0


2024-05-29 21:49:27.336555: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
class LiverLesionDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transforms=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transforms = transforms
        self.imgs = sorted(os.listdir(image_dir))
        self.annotations = set(os.path.splitext(ann)[0] for ann in os.listdir(annotation_dir))

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.imgs[idx])
        img_name = os.path.splitext(self.imgs[idx])[0]
        annotation_path = os.path.join(self.annotation_dir, f"{img_name}.txt")
        
        img = Image.open(img_path).convert("RGB")
        width, height = img.size
        
        boxes = []
        labels = []

        if img_name in self.annotations:
            with open(annotation_path) as f:
                for line in f:
                    parts = line.strip().split()
                    label = int(parts[0])
                    x_center, y_center, box_width, box_height = map(float, parts[1:])
                    
                    x_min = (x_center - box_width / 2) * width
                    y_min = (y_center - box_height / 2) * height
                    x_max = (x_center + box_width / 2) * width
                    y_max = (y_center + box_height / 2) * height

                    # Swap coordinates if necessary to ensure valid bounding boxes
                    if x_max < x_min:
                        x_min, x_max = x_max, x_min
                    if y_max < y_min:
                        y_min, y_max = y_max, y_min

                    labels.append(label)
                    boxes.append([x_min, y_min, x_max, y_max])

        else:
            labels.append(-1)  # Assuming class -1 means "normal"
            # boxes.append([0, 0, 0, 0])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        
        if self.transforms:
            img = self.transforms(img)
        
        return img, target

    def __len__(self):
        return len(self.imgs)
    
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [5]:
model_path = 'hello_resnet50.pth'
models = torch.load(model_path)
models.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [8]:
# Update the paths for your dataset
train_image_dir = 'train/train/images'
train_annotation_dir = 'train/train/annotations'
val_image_dir = 'val/val/images'
val_annotation_dir = 'val/val/annotations'

train_dataset = LiverLesionDataset(image_dir=train_image_dir, annotation_dir=train_annotation_dir, transforms=get_transform(train=True))
val_dataset = LiverLesionDataset(image_dir=val_image_dir, annotation_dir=val_annotation_dir, transforms=get_transform(train=False))

# Define data loaders
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))
val_data_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

# Get the number of input features for the classifier
in_features = models.roi_heads.box_predictor.cls_score.in_features

# Define the number of classes (7 classes + normal)
num_classes = 8  # including class -1 for "normal"

In [12]:
models.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [9]:
# Replace the pre-trained head with a new one
models.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Move model to the correct device
models.to(device)

# Define the optimizer
params = [p for p in models.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [10]:
# Training loop
num_epochs = 30

for epoch in range(num_epochs):
    models.train()
    train_loss = 0
    for images, targets in tqdm(train_data_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        # Ensure images and targets are on the correct device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Filter out targets with no bounding boxes
        valid_targets = [target for target in targets if len(target['boxes']) > 0]
        
        if not valid_targets:
            continue

        valid_images = [images[i] for i in range(len(targets)) if len(targets[i]['boxes']) > 0]
        
        loss_dict = models(valid_images, valid_targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        train_loss += losses.item()
    
    train_loss /= len(train_data_loader)
    
    # Update the learning rate
    lr_scheduler.step()
    
    # Empty CUDA cache to free up memory
    torch.cuda.empty_cache()
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

# No need to save the model to a file, the model is kept in memory
trained_model = models

Training Epoch 1/30: 100%|██████████| 452/452 [05:32<00:00,  1.36it/s]


Epoch 1, Train Loss: 0.2478


Training Epoch 2/30: 100%|██████████| 452/452 [04:52<00:00,  1.54it/s]


Epoch 2, Train Loss: 0.2322


Training Epoch 3/30: 100%|██████████| 452/452 [04:54<00:00,  1.54it/s]


...

Epoch 28, Train Loss: 0.1976


Training Epoch 29/30: 100%|██████████| 452/452 [04:54<00:00,  1.54it/s]


Epoch 29, Train Loss: 0.1990


Training Epoch 30/30: 100%|██████████| 452/452 [04:55<00:00,  1.53it/s]

Epoch 30, Train Loss: 0.1967


In [29]:
# กำหนดคลาส Dataset สำหรับชุดทดสอบที่ไม่มี ground truth
class TestDataset(Dataset):
    def __init__(self, image_dir, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms
        self.imgs = sorted(os.listdir(image_dir))

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        
        if self.transforms:
            img = self.transforms(img)
        
        return img, self.imgs[idx]  # Return image and its filename

    def __len__(self):
        return len(self.imgs)

# กำหนด transformations สำหรับชุดทดสอบ
def get_transform(train=False):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [30]:
# กำหนด path สำหรับชุดทดสอบ
test_image_dir = 'test/test/images'

# สร้าง Test Dataset และ DataLoader
test_dataset = TestDataset(image_dir=test_image_dir, transforms=get_transform(train=False))
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=lambda x: x)

def predict(model, data_loader, device, score_threshold=0.5, iou_threshold=0.3):
    model.eval()
    results = []
    with torch.no_grad():
        for images in tqdm(data_loader, desc="Predicting"):
            for image, filename in images:
                image = image.to(device)
                outputs = model([image])
                
                if len(outputs[0]['boxes']) == 0:
                    results.append((filename, -1, -1))  # No boxes detected, class -1
                else:
                    boxes = outputs[0]['boxes']
                    scores = outputs[0]['scores']
                    labels = outputs[0]['labels']
                    
                    # Apply score threshold
                    keep = scores >= score_threshold
                    boxes = boxes[keep]
                    scores = scores[keep]
                    labels = labels[keep]

                    # Apply NMS
                    keep = nms(boxes, scores, iou_threshold)
                    boxes = boxes[keep].cpu().numpy().tolist()
                    labels = labels[keep].cpu().numpy().tolist()

                    results.append((filename, boxes, labels))

    return results

In [27]:
# ทำนายผลลัพธ์บนชุดทดสอบ
test_predictions = predict(trained_model, test_data_loader, device)

submission_data = []
for filename, boxes, labels in test_predictions:
    if boxes == -1:
        submission_data.append([filename, '[]', '[]'])
    else:
        submission_data.append([filename, str(boxes), str(labels)])
submission_df = pd.DataFrame(submission_data, columns=['filename', 'boxes', 'labels'])

Predicting: 100%|██████████| 5153/5153 [06:25<00:00, 13.36it/s]


In [32]:
submission_df

,filename,boxes,labels
0,1.jpg,[],[]
1,10.jpg,[],[]
2,1000.jpg,[],[]
3,10001.jpg,[],[]
4,100077.jpg,[],[]
...,...,...,...
5148,998.jpg,[],[]
5149,99805.jpg,[],[]
5150,9982.jpg,[],[]
5151,999.jpg,"[[582.9420166015625, 920.2531127929688, 863.24...","[6, 6, 6]"


In [33]:
sample_submission = pd.read_csv('sample_submission_liver.csv')

from tqdm.notebook import tqdm, trange
for i in trange(3, submission.shape[0]):
  results = model('test/images/' + strsample_submission.iloc[i, 0]) + '.jpg', conf=0.25, verbose=False)
  sample_submission.loc[i,'Annotation'] = str(results[0].boxes.xyxy.round().cpu().numpy().astype(int).tolist())
  sample_submission.loc[i,'Label'] = str(results[0].boxes.cls.cpu().numpy().astype(int).tolist())

SyntaxError: unmatched ')' (3098031779.py, line 5)

In [36]:
# โหลด sample submission เพื่อให้ได้โครงสร้างที่ถูกต้อง
sample_submission_path = 'sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)
sample_submission

,Image File,Annotation,Label
0,18999,"[[281, 183, 339, 266]]",[1]
1,102037,"[[263, 171, 359, 263]]",[4]
2,138340,"[[85, 235, 178, 333]]",[4]
3,140589,NaN,NaN
4,20235,NaN,NaN
...,...,...,...
5148,3575,NaN,NaN
5149,190,NaN,NaN
5150,1404,NaN,NaN
5151,184,NaN,NaN


In [39]:
import pandas as pd

# Load the CSV files
sample_submission_df = pd.read_csv("sample_submission.csv")
test1_df = submission_df

# Ensure 'Image File' columns are treated as strings
sample_submission_df['Image File'] = sample_submission_df['Image File'].astype(str)
test1_df['Image File'] = test1_df['Image File'].astype(str)

# Ensure filenames in test1_df do not have extensions
test1_df['Image File'] = test1_df['Image File'].str.replace('.jpg', '')

# Perform the merge to reorder test1_df to match sample_submission_df
ordered_test1_df = sample_submission_df[['Image File']].merge(test1_df, on='Image File', how='left')

# Save the reordered test1 dataframe
ordered_test1_df_path = 'ordered_test1.csv'
ordered_test1_df.to_csv(ordered_test1_df_path, index=False)

# Display the first few rows of the reordered dataframe to verify
ordered_test1_df

,Image File,Annotation,Label
0,18999,[],[]
1,102037,[],[]
2,138340,[],[]
3,140589,"[[598.126220703125, 374.4117126464844, 658.103...",[4]
4,20235,"[[290.2486572265625, 110.16242980957031, 344.0...",[1]
...,...,...,...
5148,3575,[],[]
5149,190,[],[]
5150,1404,[],[]
5151,184,[],[]
